In [ ]:
import cv2
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import time

def custom_nms_boxes(nms_boxes, sorted_confidences, iou_threshold):
    if len(nms_boxes) == 0:
        return []

    indices = []  # List to store the indices of boxes to keep
    boxes = np.array(nms_boxes)
    confidences = np.array(sorted_confidences)

    # Extract coordinates of bounding boxes
    x1 = boxes[:, 0]  # x-coordinate of the top-left corner
    y1 = boxes[:, 1]  # y-coordinate of the top-left corner
    x2 = boxes[:, 2]  # x-coordinate of the bottom-right corner
    y2 = boxes[:, 3]  # y-coordinate of the bottom-right corner

    # Compute the area of each bounding box
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)

    # create an array of indices from 0 to N-1
    order = np.arange(len(confidences))

    while order.size > 0:
        # Index of the current box with the highest confidence score
        i = order[0]
        indices.append(i)  # Add current index to the list of kept indices
        xx1 = np.maximum(x1[i], x1[order[1:]])  # Max of top-left x-coordinates
        yy1 = np.maximum(y1[i], y1[order[1:]])  # Max of top-left y-coordinates
        xx2 = np.minimum(x2[i], x2[order[1:]])  # Min of bottom-right x-coordinates
        yy2 = np.minimum(y2[i], y2[order[1:]])  # Min of bottom-right y-coordinates

        # Compute the width and height of the intersection rectangles
        w = np.maximum(0, xx2 - xx1 + 1)  # Overlapping width
        h = np.maximum(0, yy2 - yy1 + 1)  # Overlapping height

        intersection = w * h
        union = areas[i] + areas[order[1:]] - intersection
        iou = intersection / union

        # Identify boxes with IoU less than or equal to the threshold
        inds = np.where(iou <= iou_threshold)[0]

        # Update the order array to process the next set of boxes
        order = order[inds + 1]

    return indices

def apply_nms_and_filter_by_best_scale(matching_results, template_shape, iou_threshold=0.55):
    if not matching_results:
        # print("No matching results found.")
        return [], None

    # Find the best scale by keeping only matches with the highest match value
    best_match = max(matching_results, key=lambda x: x[2])  # x[2] is the match_val
    best_scale = best_match[1]  # The scale associated with the best match

    # Filter out all matches that don't have the best scale
    best_scale_matches = [match for match in matching_results if match[1] == best_scale]

    # Prepare bounding boxes and confidence scores for NMS (using only best scale matches)
    boxes = []
    confidences = []

    for (top_left, scale, match_val) in best_scale_matches:
        h, w = int(template_shape[0] * scale), int(template_shape[1] * scale)
        box = [int(top_left[0]), int(top_left[1]), w, h]  # (x, y, width, height)
        boxes.append(box)
        confidences.append(float(match_val))  # Convert match_val to float for OpenCV

    # Sort bounding boxes by confidence scores (from highest to lowest)
    indices = sorted(range(len(confidences)), key=lambda i: confidences[i], reverse=True)
    sorted_boxes = [boxes[i] for i in indices]
    sorted_confidences = [confidences[i] for i in indices]
    sorted_best_scale_matches = [best_scale_matches[i] for i in indices]

    # Convert to required format for NMS: boxes in (x, y, x+w, y+h) format
    nms_boxes = [[x, y, x + w, y + h] for (x, y, w, h) in sorted_boxes]

    # Apply custom NMS
    nms_indices = custom_nms_boxes(nms_boxes, sorted_confidences, iou_threshold)

    # Collect the filtered results after NMS
    filtered_results = []
    if len(nms_indices) > 0:
        for i in nms_indices:
            filtered_results.append(sorted_best_scale_matches[i])

    return filtered_results, best_scale

def template_matching(template, img, scale_range=[0.8, 1.5], scale_step=0.05, threshold=0.9, border=0, match_one=False):
    """
    Perform multi-scale or single-scale template matching with a given RGBA template and RGB image.

    Args:
        template (numpy array): RGBA template image.
        img (numpy array): RGB image to perform matching on.
        scale_range (list): Range of scales as [min_scale, max_scale].
        scale_step (float): Step size for scale increments.
        threshold (float): Threshold for match similarity.

    Returns:
        matching_results (list): List of tuples containing (location, scale, match_val).
    """
    # Generate the scales based on the range and step
    scales = np.arange(scale_range[0], scale_range[1] + scale_step, scale_step)
    img_area = img.shape[0] * img.shape[1]

    # Split template into RGB and alpha channel
    b_channel, g_channel, r_channel, alpha_channel = cv2.split(template)
    template_rgb = cv2.merge((b_channel, g_channel, r_channel))
    mask = cv2.threshold(alpha_channel, 16, 255, cv2.THRESH_BINARY)[1]
    
    matching_results = []  # To store the locations of matches
    
    for scale in scales:
        # Resize template and mask for the current scale
        resized_template = cv2.resize(template_rgb, (0, 0), fx=scale, fy=scale)
        resized_mask = cv2.resize(mask, (0, 0), fx=scale, fy=scale)
        result = None
        
        template_area = resized_template.shape[0] * resized_template.shape[1]
        if match_one and template_area / img_area < 0.2: # Skip if template area is less than 50% of image area
            continue
        
        # Ensure the resized template is not larger than the image
        if resized_template.shape[0] > img.shape[0] or resized_template.shape[1] > img.shape[1]:
            if resized_template.shape[0] >= img.shape[0] - 2*border and resized_template.shape[1] >= img.shape[1] - 2*border:
                # perform reverse matching
                padding = 5
                img_h, img_w = img.shape[:2]
                img_without_border = img[border+padding:img_h-border-padding, border+padding:img_w-border-padding]
                result = cv2.matchTemplate(resized_template, img_without_border, cv2.TM_CCORR_NORMED)
            else:
                continue
        else:
            # Perform template matching
            result = cv2.matchTemplate(img, resized_template, cv2.TM_CCORR_NORMED, mask=resized_mask)
        
        # Find locations where the match is greater than the threshold
        loc = np.where(result >= threshold)

        # Collect all the matching points
        for pt in zip(*loc[::-1]):  # Switch x and y in zip
            matching_results.append((pt, scale, result[pt[1], pt[0]]))
    return matching_results

def process_template_matches(template_match_data, template_dir, img, iou_threshold=0.1, scale_range=[0.9, 1.3], scale_step=0.05, threshold=0.9, min_area=1000, match_one=False, border=0, debug=False):
    """
    Perform template matching for multiple templates from a folder, apply NMS and best scale filtering.

    Args:
        template_match_data (dict): Dictionary to store matching results and best scales, with template IDs as keys.
        template_dir (Path): Directory containing template images.
        img (ndarray): target image.
        iou_threshold (float): IoU threshold for NMS.
        scale_range (list): Range of scales as [min_scale, max_scale].
        scale_step (float): Step size for scale increments.
        threshold (float): Threshold for match similarity.
        min_area (int): Minimum area of the bounding box to keep.
        match_one (bool): Set true to return the highest score match.
        border (int): Border size to ignore around the image.
    Returns:
        match_one_result (str): The template ID of the first match found, if match_one is True.
    """
    max_score = 0
    match_one_template = None
    match_one_template_shape = None
    match_one_filtered_results = None
    match_one_scale = None
    
    # Iterate through each template in the folder
    for path in template_dir.glob('*.png'):  # Assuming templates are PNG files
        template_name = path.stem
        template = cv2.imread(str(path), cv2.IMREAD_UNCHANGED)  # Load template as RGBA
        template_shape = template.shape  # Get template shape for NMS

        # Check if this template already has a best scale in the template_match_data
        if template_name in template_match_data and template_match_data[template_name]['best_scale'] is not None:
            best_scale = template_match_data[template_name]['best_scale']
            matching_results = template_matching(template, img, scale_range=[best_scale, best_scale], scale_step=1.0, threshold=threshold, border=border, match_one=match_one)
        else:
            matching_results = template_matching(template, img, scale_range=scale_range, scale_step=scale_step, threshold=threshold, border=border, match_one=match_one)
        
        # Apply NMS and filter by best scale
        filtered_results, best_scale = apply_nms_and_filter_by_best_scale(matching_results, template_shape, iou_threshold=iou_threshold)
        
        # Check if the area of the bounding box is less than the minimum area
        if (best_scale is not None) and (template_shape[0] * best_scale) * (template_shape[1] * best_scale) < min_area:
            best_scale = None
            filtered_results = []
            
        # Skip if no matches found
        if best_scale is None or not filtered_results:
            continue
        
        # Add or update the result in the dictionary
        if not match_one:
            if template_name not in template_match_data:
                template_match_data[template_name] = {
                    'shape': template_shape,
                    'result': filtered_results,
                    'best_scale': best_scale
                }
            else:
                template_match_data[template_name]['result'] = filtered_results
        
        elif match_one:
            if max_score < filtered_results[0][2]:
                if debug:
                    print(f"Match found for {template_name} with score {filtered_results[0][2]}")
                max_score = filtered_results[0][2]
                match_one_template = template_name
                match_one_template_shape = template_shape
                match_one_filtered_results = filtered_results
                match_one_scale = best_scale

        
    if match_one == True:
        template_match_data[match_one_template] = {
            'shape': match_one_template_shape,
            'result': match_one_filtered_results,
            'best_scale': match_one_scale
        }
        return match_one_template

def draw_bboxes_and_icons_on_image(img, template_dir, grid, icon_size=50):
    color = (255, 255, 255)
    
    for i in range(grid.row):
        for j in range(grid.col):
            if grid[i, j] is None:
                continue
            template_name = grid[i, j]
            template_path = template_dir / f"{template_name}.png"
            template = cv2.imread(str(template_path), cv2.IMREAD_UNCHANGED)
            template_rgb = cv2.cvtColor(template, cv2.COLOR_BGRA2BGR)  # Convert to RGB, discard alpha
            resized_template = cv2.resize(template_rgb, (icon_size, icon_size))
            
            # Draw the bounding box
            x, y, w, h = grid.get_roi(i, j)
            cv2.rectangle(img, (x, y), (x+w, y+h), color, 5)
            
            icon_top_left = (x, y)
            icon_bottom_right = (x + resized_template.shape[1], y + resized_template.shape[0])
            img[icon_top_left[1]:icon_bottom_right[1], icon_top_left[0]:icon_bottom_right[0]] = resized_template
    
    # Convert BGR (OpenCV) image to RGB for displaying with plt
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Show the image with the bounding boxes and template icons
    plt.figure(figsize=(10, 10))
    plt.imshow(img_rgb)
    plt.axis('off')  # Hide axes
    plt.show()
    
def get_grid_info(points,tolerance=30):
    # tolerance is in pixel
    def get_cluster_center(values):       
        grouped_indices = set()
        cluster_center = []
        
        while True:
            reference_values = []
            for i in range(len(values)):
                if i in grouped_indices:
                    continue
                reference_values.append(values[i])
                grouped_indices.add(i)
                break
            else: break
            
            grouping_completed = False
            while not grouping_completed:
                grouping_completed = True
                i = 0
                while i < len(reference_values):
                    for j in range(len(values)):
                        if j not in grouped_indices and abs(reference_values[i] - values[j]) < tolerance:
                            reference_values.append(values[j])
                            grouped_indices.add(j)
                            grouping_completed = False
                    i += 1

            cluster_center.append(sum(reference_values) / len(reference_values))

        cluster_center.sort()
        return cluster_center
    
    col_x = get_cluster_center([point[0] for point in points])
    row_y = get_cluster_center([point[1] for point in points])

    col_width = []
    for i in range(len(col_x)-1):
        col_width.append(col_x[i+1] - col_x[i])
    grouped_col_width = min(get_cluster_center(col_width))

    avg_col_width = grouped_col_width
    
    row_height = []
    for i in range(len(row_y)-1):
        row_height.append(row_y[i+1] - row_y[i])
    avg_row_height = min(get_cluster_center(row_height))

    display_x      = int(min(col_x) - avg_col_width/2)
    display_y      = int(min(row_y) - avg_row_height/2)
    display_width  = int(max(col_x) + avg_col_width/2 - display_x)
    display_height = int(max(row_y) + avg_row_height/2 - display_y)
    m = round((max(row_y)-min(row_y))/avg_row_height+1)
    n = round((max(col_x)-min(col_x))/avg_col_width+1)
    
    return (display_x, display_y, display_width, display_height), (m,n)

class BaseGrid:
    def __init__(self, bbox, shape):
        self.bbox = bbox
        self.row, self.col = shape
        x, y, w, h = bbox
        self.symbol_width = w // self.col
        self.symbol_height = h // self.row
        self._grid = [[None for _ in range(self.col)] for _ in range(self.row)]
        
    def get_roi(self, i, j):
        x, y, w, h = self.bbox
        return (int(x + self.symbol_width * j), int(y + self.symbol_height * i), int(self.symbol_width), int(self.symbol_height))
    
    def clear(self):
        for i in range(self.row):
            for j in range(self.col):
                self._grid[i][j] = None
    
    def __getitem__(self, idx):
        i, j = idx
        return self._grid[i][j]

    def __setitem__(self, idx, value):
        i, j = idx
        self._grid[i][j] = value
    
class BullGrid(BaseGrid):
    def __init__(self, bbox, shape, growth_direction='up'):
        super().__init__(bbox, shape)
        self.growth_direction = growth_direction  # To record the current growth direction
        self.column_heights = [self.row] * self.col  # Track the heights of each column
        self.base_height = 3
        self.max_height = 7
    
    def init_column_heights(self):
        # Initialize the column heights based on the growth direction
        if self.growth_direction == 'up':
            for j in range(self.col):
                for i in range(self.row):
                    if self._grid[i][j] is not None:
                        self.column_heights[j] = self.row - i
                        break
        elif self.growth_direction == 'down':
            for j in range(self.col):
                for i in range(self.row-1, -1, -1):
                    if self._grid[i][j] is not None:
                        self.column_heights[j] = i + 1
                        break

    def add_row(self, position='top'):
        x, y, w, h = self.bbox
        self.row += 1

        # Adjust the bounding box and grid based on the position
        if position == 'top':
            y -= self.symbol_height
            h += self.symbol_height
            self.bbox = (x, y, w, h)
            self._grid.insert(0, [None for _ in range(self.col)])
        elif position == 'bottom':
            h += self.symbol_height
            self.bbox = (x, y, w, h)
            self._grid.append([None for _ in range(self.col)])
        else:
            raise ValueError("position must be 'top' or 'bottom'")

In [ ]:
template_dir = Path('./images/bull/symbols/base_game')
image_dir = Path('./images/bull/base_game')

template_match_data = {}
grid = None
is_init_column_heights = False

up_arrow_name = "up_arrow"
up_arrow_template = cv2.imread(str(template_dir / f'{up_arrow_name}.png'), cv2.IMREAD_UNCHANGED)
up_arrow_scale_range = [1.0, 2.0]


cell_border = 10

skip = 0

for image_path in image_dir.glob('*.png'):
    if skip > 0:
        skip -= 1
        continue
    
    print(f"Processing image: {image_path}")
    img = cv2.imread(str(image_path))
    
    # initialize grid
    if grid is None:
        start_time = time.time()
        process_template_matches(
            template_match_data=template_match_data, 
            template_dir=template_dir, 
            img=img, 
            iou_threshold=0.1, 
            scale_range=[0.9, 1.3],
            threshold=0.95,
            min_area=5000,
            border=100
        )
        elapsed_time = time.time() - start_time
        print(f"Initial grid matching: {elapsed_time:.2f} seconds")
        
        matched_positions = []
        for template_name, data in template_match_data.items():
            w, h = data['shape'][1], data['shape'][0]
            for (top_left, scale, _) in data['result']:
                x = top_left[0] + w * scale / 2
                y = top_left[1] + h * scale / 2
                matched_positions.append((x, y))
        if len(matched_positions) == 0:
            print("Could not find any matches")
            break
                
        grid_bbox, grid_shape = get_grid_info(matched_positions)
        grid = BullGrid(grid_bbox, grid_shape, 'up')
        print(f'initial grid shape: {grid.row} x {grid.col}')
    
    
    # Process each grid cell
    start_time = time.time()
    if grid.growth_direction == 'up':
        for j in range(grid.col):
            for i in range(grid.row - grid.column_heights[j], grid.row):
                roi = grid.get_roi(i, j)
                x, y, w, h = roi
                x1 = x-cell_border
                x2 = x+w+cell_border
                y1 = y-cell_border
                y2 = y+h+cell_border
                cell = img[y1:y2, x1:x2]
                symbol_name = process_template_matches(
                    template_match_data=template_match_data, 
                    template_dir=template_dir, 
                    img=cell, 
                    iou_threshold=0.1, 
                    scale_range=[0.9, 1.3],
                    threshold=0.9,
                    min_area=5000,
                    match_one=True,
                    border=cell_border
                )
                grid[i, j] = symbol_name
    
    if not is_init_column_heights:
        grid.init_column_heights()
        is_init_column_heights = True
        print("initial column heights:", grid.column_heights)
        
    elapsed_time = time.time() - start_time
    print(f"Grid cells matching: {elapsed_time:.2f} seconds")
    
    #find arrow at each column
    start_time = time.time()
    for j in range(grid.col):
        if grid.growth_direction == 'up':
            roi = grid.get_roi(grid.row-grid.column_heights[j]-1, j)
            x, y, w, h = roi
            match_result = template_matching(up_arrow_template, img[y:y+h, x:x+w], scale_range=up_arrow_scale_range, threshold=0.90)
            match_result, best_scale = apply_nms_and_filter_by_best_scale(match_result, up_arrow_template.shape, iou_threshold=0.1)
            if len(match_result) > 0:
                up_arrow_scale_range = [best_scale, best_scale]
                grid.column_heights[j] += 1
                if grid.column_heights[j] > grid.max_height:
                    grid.column_heights[j] = grid.base_height
                print(f'up arrow found at column[{j}], update column_heights[{j}] to {grid.column_heights[j]}')
                if grid.column_heights[j] > grid.row:
                    grid.add_row(position='top')
                    print(f'Updated grid shape: {grid.row} x {grid.col}')
    for j in range(grid.col):
        print(f'column[{j}] height: {grid.column_heights[j]}')
    
    draw_bboxes_and_icons_on_image(img, template_dir, grid, 50)
    
    grid.clear()
    
    elapsed_time = time.time() - start_time
    print(f"Up arrow matching: {elapsed_time:.2f} seconds")
    print("------------------------------------------------------")

In [ ]:
template_dir = Path('./images/bull/symbols/free_game')
image_dir = Path('./images/bull/free_game')

template_match_data = {}
grid = None
is_init_column_heights = False

down_arrow_name = "down_arrow"
down_arrow_template = cv2.imread(str(template_dir / f'{down_arrow_name}.png'), cv2.IMREAD_UNCHANGED)
down_arrow_scale_range = [1.0, 2.0]

cell_border = 10

skip = 0

for image_path in image_dir.glob('*.png'):
    if skip > 0:
        skip -= 1
        continue
    
    print(f"Processing image: {image_path}")
    img = cv2.imread(str(image_path))
    
    # initialize grid
    if grid is None:
        start_time = time.time()
        process_template_matches(
            template_match_data=template_match_data, 
            template_dir=template_dir, 
            img=img, 
            iou_threshold=0.1, 
            scale_range=[0.9, 1.5],
            threshold=0.9,
            min_area=5000,
            border=100
        )
        elapsed_time = time.time() - start_time
        print(f"Initial grid matching: {elapsed_time:.2f} seconds")
        
        img_copy = img.copy()
        matched_positions = []
        for template_name, data in template_match_data.items():
            w, h = data['shape'][1], data['shape'][0]
            for (top_left, scale, _) in data['result']:
                x = top_left[0] + w * scale / 2
                y = top_left[1] + h * scale / 2
                matched_positions.append((x, y))
                cv2.circle(img_copy, (int(x), int(y)), 5, (0, 0, 255), -1)
            # print(f"Template: {template_name}, Matches: {len(data['result'])}")
        # img_copy = cv2.resize(img_copy, dsize=(0, 0), fx=0.5, fy=0.5)
        # cv2.imshow('init match', img_copy)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        if len(matched_positions) == 0:
            print("Could not find any matches")
            break
        
        grid_bbox, grid_shape = get_grid_info(matched_positions)
        grid = BullGrid(grid_bbox, grid_shape, 'down')
        # template_match_data = {}
    
    
    # Process each grid cell
    start_time = time.time()
    if grid.growth_direction == 'down':
        for j in range(grid.col):
            for i in range(grid.column_heights[j]):
                roi = grid.get_roi(i, j)
                x, y, w, h = roi
                x1 = x-cell_border
                x2 = x+w+cell_border
                y1 = y-cell_border
                y2 = y+h+cell_border
                cell = img[y1:y2, x1:x2]
                symbol_name = process_template_matches(
                    template_match_data=template_match_data, 
                    template_dir=template_dir, 
                    img=cell, 
                    iou_threshold=0.1, 
                    scale_range=[0.9, 1.3],
                    threshold=0.9,
                    min_area=5000,
                    match_one=True,
                    border=cell_border,
                )
                grid[i, j] = symbol_name
    elapsed_time = time.time() - start_time
    print(f"Grid cells matching: {elapsed_time:.2f} seconds")
                
    #find arrow at each column
    if is_init_column_heights == False:
        grid.init_column_heights()
        is_init_column_heights = True
        
    start_time = time.time()
    for j in range(grid.col):
        if grid.growth_direction == 'up':
            roi = grid.get_roi(grid .row-grid.column_heights[j]-1, j)
            x, y, w, h = roi
            match_result = template_matching(up_arrow_template, img[y:y+h, x:x+w], scale_range=up_arrow_scale_range, threshold=0.90)
            match_result, best_scale = apply_nms_and_filter_by_best_scale(match_result, up_arrow_template.shape, iou_threshold=0.1)
            if len(match_result) > 0:
                up_arrow_scale_range = [best_scale, best_scale]
                grid.column_heights[j] += 1
                if grid.column_heights[j] > grid.max_height:
                    grid.column_heights[j] = grid.base_height
                print(f'up arrow found at column[{j}], update column_heights[{j}] to {grid.column_heights[j]}')
                if grid.column_heights[j] > grid.row:
                    grid.add_row(position='top')
                    print(f'Updated grid shape: {grid.row} x {grid.col}')
        elif grid.growth_direction == 'down':
            roi = grid.get_roi(grid.column_heights[j], j)
            x, y, w, h = roi
            match_result = template_matching(down_arrow_template, img[y:y+h, x:x+w], scale_range=down_arrow_scale_range, threshold=0.90)
            match_result, best_scale = apply_nms_and_filter_by_best_scale(match_result, down_arrow_template.shape, iou_threshold=0.1)
            if len(match_result) > 0:
                down_arrow_scale_range = [best_scale, best_scale]
                grid.column_heights[j] += 1
                if grid.column_heights[j] > grid.max_height:
                    grid.column_heights[j] = grid.base_height
                print(f'Down arrow found at column[{j}], update column_heights[{j}] to {grid.column_heights[j]}')
                if grid.column_heights[j] > grid.row:
                    grid.add_row(position='bottom') 
                    print(f'Updated grid shape: {grid.row} x {grid.col}')
    for j in range(grid.col):
        print(f'column[{j}] height: {grid.column_heights[j]}')
        
    
    
    draw_bboxes_and_icons_on_image(img, template_dir, grid, 50)
    
    
    grid.clear()
    
    elapsed_time = time.time() - start_time
    print(f"Up arrow matching: {elapsed_time:.2f} seconds")
    print("------------------------------------------------------")

In [ ]:
template_dir = Path('./images/bull/symbols/free_game')
image_dir = Path('./temp')

template_match_data = {}
grid = None
border = 10
scale_range = [0.5, 5.0]

for image_path in image_dir.glob('*.png'):
    img = cv2.imread(str(image_path))
    print(f'img area: {img.shape[0] * img.shape[1]}')
    for template_path in template_dir.glob('*.png'):
        if template_path.stem != "ingot":
            continue
        template = cv2.imread(str(template_path), cv2.IMREAD_UNCHANGED)
        template_shape = template.shape

        matching_results = template_matching(template, img, scale_range=scale_range, threshold=0.8, border=border)
        filtered_results, best_scale = apply_nms_and_filter_by_best_scale(matching_results, template_shape, iou_threshold=0.1)
        for res in filtered_results:
            print(res[1], res[2])
        print(f'best_scale: {best_scale}')
        print(f'template area: {template_shape[0] * best_scale * template_shape[1] * best_scale}')
        template = cv2.resize(template, (0, 0), fx=best_scale, fy=best_scale)
        cv2.imshow('template', template)
        cv2.imshow('cell', img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()